In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

%matplotlib inline

Chargement des données

In [2]:
output_conv1 = np.load('Outputs/output_conv1.npy')
output_conv1_br = np.load('Outputs/output_conv1_br.npy')

weights = np.load('parameters/vgg16_weights.npz')

Entrées de l'algorithme

In [3]:
output = output_conv1_br.reshape(224*224,64)[:100].reshape(10,10,64)
Outputs = [output,output]

In [4]:
W_w = weights['conv1_1_W']
W_b = weights['conv1_1_b']

Cas non-linéaire

In [5]:
d_prime = 10

In [6]:
def GSVD(A,Y):
    M = np.eye(A.shape[0])
    
    U,s,V = np.linalg.svd(Y)
    sqrtYYT = U.dot(np.diag(s)).dot(U.transpose())
    
    A_tilde = M.dot(A).dot(sqrtYYT)
    U,s,V = np.linalg.svd(A_tilde,full_matrices=True)
    
    return M.dot(U), s, np.linalg.inv(sqrtYYT).dot(V.transpose())

In [7]:
k = W_w.shape[0]
c = W_w.shape[2]
d = W_w.shape[3]
n = Outputs[0].shape[0]

Y = Outputs[0].reshape(n*n,d).transpose()
for i in range(1,len(Outputs)):
    Y = np.concatenate((Y,Outputs[i].reshape(n*n,d).transpose()),axis=1)
Y_bar = np.mean(Y,axis=1,keepdims=True)
Y = Y - Y_bar

Z = np.zeros(Y.shape)

In [8]:
lambd = 0.01
for it in range(5):
    # 25 iterations with 0.01, 25 with 1 (like in Zhang et al.)
    if it > 24:
        lambd=1
    
    # Minimization on M, b

    Z_bar = np.mean(Z,axis=1,keepdims=True)
    M_hat = Z.dot(Y.transpose()).dot(np.linalg.inv(Y.dot(Y.transpose())))
    U,s,V = GSVD(M_hat,Y)
    
    M = U.T[:d_prime].T.dot(np.diag(s[:d_prime])).dot(V.T[:d_prime])
    b = (Z_bar - M.dot(Y_bar)).reshape(d)

    # Minimization on Z

    Y_prime = M.dot(Y) + b.reshape(64,1)
    Z_0 = np.minimum(Y_prime,0)
    Z_1 = np.maximum((lambd*Y_prime + np.maximum(Y,0))/(1+lambd),0)

    for i in range(d):
        for j in range(n*n*len(Outputs)):
            q0 = (np.max(Y[i][j],0)-np.max(Z_0[i][j],0))**2 + lambd*(Z_0[i][j]-Y_prime[i][j])**2
            q1 = (np.max(Y[i][j],0)-np.max(Z_1[i][j],0))**2 + lambd*(Z_1[i][j]-Y_prime[i][j])**2
            if q0 < q1:
                Z[i][j] = Z_0[i][j]
            else:
                Z[i][j] = Z_1[i][j]
                
    print("Objective :",np.sum((np.maximum(Y,0) - np.maximum(M.dot(Y) + b.reshape(64,1),0))**2))

Objective : 1186276.10036
Objective : 1.31401325961e+15
Objective : 7.37589129585e+21
Objective : 4.13675815842e+28
Objective : 2.32009403307e+35


TESTS : Fonction GSVD

In [9]:
A = M_hat
M = np.eye(A.shape[0])

U_Y,s_Y,V_Y = np.linalg.svd(Y)
s_mod = [s_Y[i] if i < len(s_Y) else 0 for i in range(Y.shape[0])]

YYT = U_Y.dot(np.diag(s_mod)**2).dot(U_Y.transpose())
sqrtYYT = U_Y.dot(np.diag(s_mod)).dot(U_Y.transpose())

A_tilde = M.dot(A).dot(sqrtYYT)
U_SVD,s_SVD,V_SVD = np.linalg.svd(A_tilde,full_matrices=True)

print("Test matrix inversion :",np.linalg.norm(np.linalg.inv(sqrtYYT).dot(sqrtYYT) - np.eye(d),1),'(should be 0)')
U,s,V = M.dot(U_SVD), s_SVD, np.linalg.inv(sqrtYYT).dot(V_SVD.transpose())
print("Test result :",np.linalg.norm(V.dot(YYT).dot(V.transpose()) - np.eye(d),1),'(should be 0)')

Test matrix inversion : 13400.5488281 (should be 0)
Test result : 1.20479503598e+26 (should be 0)
